## Filtering a Data Observatory dataset using "Who's On First" in CARTOFrames

This notebook illustrates how to use the admin. region geometries from Who's on First (public data) to filter a dataset from CARTO's [Data Observatory](https://carto.com/spatial-data-catalog/) using [CARTOFrames](https://carto.com/cartoframes/) methods.

The notebook is organized as follows:
0. Setup account
1. Access a dataset from a Data Observatory subscription to be filtered
2. Who's on First for filtering data in cities

**Documentation**
- CARTO Spatial Data Catalogue - [link](https://carto.com/spatial-data-catalog/browser/)
- CARTOFrames technical documentation - [link](https://carto.com/developers/cartoframes/)
- "Who's on First" GeoJSON data product - [link](https://carto.com/spatial-data-catalog/browser/geography/wof_geojson_4e78587c/data)

### 0. Setup

In [1]:
import geopandas as gpd
import pandas as pd

from cartoframes.auth import set_default_credentials
from cartoframes.data.observatory import *
from cartoframes.viz import *

pd.set_option('display.max_columns', None)

In [2]:
set_default_credentials('creds.json')

### 1. Access a dataset from a Data Observatory subscription to be filtered

First, we check our data subscriptions from the Data Observatory to select which dataset we want to filter.

In [3]:
Catalog().subscriptions().datasets.to_dataframe()

,slug,name,description,category_id,country_id,data_source_id,provider_id,geography_name,geography_description,temporal_aggregation,time_coverage,update_frequency,is_public_data,lang,version,category_name,provider_name,geography_id,id
0,ags_sociodemogr_a7e14220,Sociodemographics - United States of America (...,Census and ACS sociodemographic data estimated...,demographics,usa,sociodemographics,ags,Census Block Group - United States of America,None,yearly,None,None,False,eng,2020,Demographics,Applied Geographic Solutions,carto-do.ags.geography_usa_blockgroup_2015,carto-do.ags.demographics_sociodemographics_us...
1,ags_retailpoten_aaf25a8c,Retail Potential - United States of America (C...,The retail potential database consists of aver...,demographics,usa,retailpotential,ags,Census Block Group - United States of America ...,Shoreline clipped TIGER/Line boundaries. More ...,yearly,"[2018-01-01, 2019-01-01)",None,False,eng,2019,Demographics,Applied Geographic Solutions,carto-do-public-data.carto.geography_usa_block...,carto-do.ags.demographics_retailpotential_usa_...
2,pb_consumer_po_62cddc04,Points Of Interest - Consumer - United States ...,Consumer Point of interest database per catego...,points_of_interest,usa,consumer_points_of_interest,pitney_bowes,Latitude/Longitude - United States of America,Location of Points of Interest,monthly,None,monthly,False,eng,v1,Points of Interest,Pitney Bowes,carto-do.pitney_bowes.geography_usa_latlon_v1,carto-do.pitney_bowes.pointsofinterest_consume...
3,ags_sociodemogr_f510a947,Sociodemographics - United States of America (...,Census and ACS sociodemographic data estimated...,demographics,usa,sociodemographics,ags,Census Block Group - United States of America ...,Shoreline clipped TIGER/Line boundaries. More ...,yearly,"[2019-01-01, 2020-01-01)",None,False,eng,2019,Demographics,Applied Geographic Solutions,carto-do-public-data.carto.geography_usa_block...,carto-do.ags.demographics_sociodemographics_us...
4,ags_consumer_sp_dbabddfb,Consumer Spending - United States of America (...,The Consumer Expenditure database consists of ...,demographics,usa,consumer_spending,ags,Census Block Group - United States of America,None,yearly,None,None,False,eng,2020,Demographics,Applied Geographic Solutions,carto-do.ags.geography_usa_blockgroup_2015,carto-do.ags.demographics_consumerspending_usa...
5,spa_geosocial_s_d5dc42ae,Geosocial Segments - United States of America ...,"By analysing feeds from Twitter, Instagram, Me...",behavioral,usa,geosocial_segments,spatial_ai,Census Block Group - United States of America ...,Shoreline clipped TIGER/Line boundaries. More ...,quarterly,"[2020-01-01, 2020-04-01)",quarterly,False,eng,v1,Behavioral,Spatial.ai,carto-do-public-data.carto.geography_usa_block...,carto-do.spatial_ai.behavioral_geosocialsegmen...
6,mc_geographic__7980c5c3,Geographic Insights - United States of America...,"Geographic Insights validate, evaluate and ben...",financial,usa,geographic_insights,mastercard,Census Block Group - United States of America ...,Shoreline clipped TIGER/Line boundaries. More ...,monthly,"[2019-01-01, 2020-01-01)",monthly,False,eng,v1,Financial,Mastercard,carto-do-public-data.carto.geography_usa_block...,carto-do.mastercard.financial_geographicinsigh...
7,pb_points_of_i_94bda91b,Points Of Interest - Spain (Latitude/Longitude),Point of interest database per categories,points_of_interest,esp,points_of_interest,pitney_bowes,Latitude/Longitude - Spain,Location of Points of Interest,monthly,None,monthly,False,eng,v1,Points of Interest,Pitney Bowes,carto-do.pitney_bowes.geography_esp_latlon_v1,carto-do.pitney_bowes.pointsofinterest_pointso...
8,u360_sociodemogr_28e93b81,Sociodemographics - Spain (Grid 100m),Curation of socio-demographic and socio-econom...,demographics,esp,sociodemographics,unica360,Grid 100m,100x100m grid,yearly,None,None,False,eng,2019,Demographics,Unica360,carto-do.unica360.geography_esp_grid100x100m_2019,carto-do.unica360.demographics_sociodemographi...


We identify the slug_id from the dataset we want to use. For example this one:

In [4]:
SpatialFeatures_esp_qk15 = Dataset.get('cdb_spatial_fea_d23a5c97')

In [5]:
SpatialFeatures_esp_qk15.to_dict()

{'slug': 'cdb_spatial_fea_d23a5c97',
 'name': 'Spatial Features - Spain (Quadgrid 15)',
 'description': 'Spatial Features is a dataset curated by CARTO providing access to a set of location-based features with global coverage that have been unified in common geographic supports (eg. Quadgrid). This product has been specially designed to facilitate spatial modeling at scale.\nSpatial Features includes core demographic data and POI aggregations by category that have been generated by processing and unifying globally available sources such as Worldpop and OpenStreetMap.\nThe current version of this product is available in two different spatial aggregations: Quadgrid level 15 (with cells of approximately 1x1km) and Quadgrid level 18 (with cells of approximately 100x100m).',
 'category_id': 'derived',
 'country_id': 'esp',
 'data_source_id': 'spatial_features',
 'provider_id': 'carto',
 'geography_name': 'Quadgrid 15 - Spain',
 'geography_description': 'Global Quadgrid (zoom level 15)',
 't

In [6]:
SpatialFeatures_esp_qk15.tail()

,male,geoid,female,retail,leisure,tourism,education,financial,food_drink,healthcare,population,country_iso,male_1_to_4,male_5_to_9,male_under_1,female_1_to_4,female_5_to_9,male_10_to_14,male_15_to_19,male_20_to_24,male_25_to_29,male_30_to_34,male_35_to_39,male_40_to_44,male_45_to_49,male_50_to_54,male_55_to_59,male_60_to_64,male_65_to_69,male_70_to_74,male_75_to_79,country_iso_a3,female_under_1,transportation,female_10_to_14,female_15_to_19,female_20_to_24,female_25_to_29,female_30_to_34,female_35_to_39,female_40_to_44,female_45_to_49,female_50_to_54,female_55_to_59,female_60_to_64,female_65_to_69,female_70_to_74,female_75_to_79,male_80_and_over,female_80_and_over
0,3157.484307,031332122022031,3314.603374,202,2,30,3,20,217,17,6987.135426,Spain,120.913153,163.556262,30.859980,112.445169,152.122509,179.977780,169.691305,162.016885,168.189652,180.966714,220.835550,269.261748,263.299398,248.028995,227.292451,196.722677,161.299894,142.868832,105.937552,ESP,28.693915,62,169.153640,158.909087,153.972406,164.477591,177.747316,219.014149,267.756015,268.073208,256.767236,237.604333,208.339251,177.525427,167.336132,139.422190,145.765479,255.243802
1,3100.709744,031333212123113,3112.707666,397,7,18,13,29,227,41,6213.417445,Spain,116.135321,156.329847,29.640561,107.896359,146.732161,171.439127,162.141215,156.255203,163.785669,177.375590,216.347498,264.268170,258.871424,245.878540,227.791271,199.504164,161.553366,141.963759,104.895240,ESP,27.533142,115,163.738535,153.326000,147.104910,155.585855,167.001699,205.922699,251.286604,251.263774,238.742479,218.524946,189.368642,163.730100,155.843235,130.658199,146.533779,238.448327
2,5563.234994,031333330303331,5634.263423,267,5,33,10,32,230,39,11197.498466,Spain,207.438940,282.454383,52.943473,196.299369,263.708241,310.187053,294.944600,277.136621,291.315402,318.226120,392.201611,481.832879,471.522448,446.028405,404.935209,345.842258,282.652461,251.291441,187.960022,ESP,50.091947,124,293.852755,273.574052,269.563257,284.239360,302.392749,368.791786,447.273132,447.816753,427.330467,399.392763,354.964127,303.556543,285.400862,236.542165,264.321666,429.473098
3,5752.487674,033121230020013,5897.919277,392,9,19,22,41,239,46,13530.062792,Spain,214.959833,292.499451,54.862983,205.108661,275.753969,323.571403,298.909093,289.252283,296.326437,318.596845,400.574748,495.122733,488.513948,459.693972,418.807097,358.926419,296.731579,266.983096,198.371388,ESP,52.339912,54,304.900184,292.604586,279.560116,302.507961,327.124385,391.198812,471.563116,468.010031,448.989864,418.053752,370.225668,313.187995,291.416965,243.300764,279.784366,442.072536
4,4920.401140,033110331211321,5101.838268,386,12,24,11,17,240,38,10022.239175,Spain,185.931533,251.828674,47.454258,175.431490,237.010205,277.479072,262.715527,251.839640,260.796087,279.359516,345.971107,422.165983,413.846829,389.538337,356.188780,306.779508,250.842350,222.981889,164.327410,ESP,44.766853,49,263.162347,246.132949,237.480096,254.350030,276.120897,335.150496,409.423620,409.001029,392.155128,363.719101,320.472125,273.839742,257.380644,215.620176,230.354640,390.621340
5,10488.067253,120222233002302,10980.102434,206,5,39,9,19,240,29,21468.169211,Spain,399.577618,543.086551,101.981938,374.481191,504.032327,597.000008,565.179574,533.200418,543.255342,585.150797,735.436253,911.480321,896.233391,834.498850,751.953428,646.956267,529.950073,474.194427,357.228428,ESP,95.560624,172,561.082684,524.800891,514.948448,560.215039,604.717769,723.562416,869.828768,866.350490,839.930112,790.127512,696.650138,593.946884,554.767677,456.639502,481.703569,848.459962
6,10691.776215,120222233002303,11193.368209,144,6,44,13,30,240,40,21885.144834,Spain,407.338614,553.634872,103.962726,381.754702,513.822090,608.595502,576.157012,543.556738,553.806953,596.516153,749.720600,929.183921,913.640858,850.707247,766.558575,659.522022,540.243248,483.404662,364.166840,ESP,97.416694,193,571.980562,534.994066,524.950252,571.096036,616.463167,737.616141,886.723386,883.177559,856.244038,805.47409

### 2. Who's On First GeoJSON for filtering data in cities

CARTO's Data Observatory also provides direct access to a group of public datasets. You can navigate and explore our Spatial Data Catalog from within your Python notebook with the Data Discovery methods in CARTOFrames or using our [Spatial Data Catalog](https://carto.com/spatial-data-catalog/browser/?license=public).

["Who's on First"](https://whosonfirst.org/) is a gazetteer (o big list) of places, each with a stable identifier and some number of descriptive properties about that location. 


We can use the WoF GeoJSON to find the city boundaries to use then for filtering the data from other datasets from the Data Observatory.

In [7]:
Catalog().provider('whos_on_first').public().geographies.to_dataframe()

You can find more entities with the Global country filter. To apply that filter run:
	Catalog().country('glo')


,slug,name,description,country_id,provider_id,geom_type,geom_coverage,update_frequency,is_public_data,lang,version,provider_name,id
0,wof_ancestors_eaaeac75,Ancestors - Global,A normalized view of the hierarchies in 'geojs...,glo,whos_on_first,MULTIPLE,None,None,True,eng,20190520,Who's On First,carto-do-public-data.whos_on_first.geography_g...
1,wof_concordance_392f80ad,Concordances - Global,Relationship between Who's On First identifier...,glo,whos_on_first,MULTIPLE,None,None,True,eng,20190520,Who's On First,carto-do-public-data.whos_on_first.geography_g...
2,wof_geojson_4e78587c,GeoJSON - Global,The main table in Who's On First. Holds all th...,glo,whos_on_first,MULTIPLE,None,None,True,eng,20190520,Who's On First,carto-do-public-data.whos_on_first.geography_g...
3,wof_names_5a30fa98,Names - Global,What things are called in Who's On First. A no...,glo,whos_on_first,MULTIPLE,None,None,True,eng,20190520,Who's On First,carto-do-public-data.whos_on_first.geography_g...
4,wof_spr_850ad7e9,Standard Places Response - Global,"The ""Standard Places Response"" (or SPR) is an ...",glo,whos_on_first,MULTIPLE,None,None,True,eng,20190520,Who's On First,carto-do-public-data.whos_on_first.geography_g...


Note that the ID to access the WoF GeoJSON table is 'wof_geojson_4e78587c'.

In [8]:
wof_geojson = Geography.get('wof_geojson_4e78587c')

In [9]:
wof_geojson.to_dict()

{'slug': 'wof_geojson_4e78587c',
 'name': 'GeoJSON - Global',
 'description': "The main table in Who's On First. Holds all the relevant information for a place in the 'body' JSON field.",
 'country_id': 'glo',
 'provider_id': 'whos_on_first',
 'geom_type': 'MULTIPLE',
 'update_frequency': None,
 'is_public_data': True,
 'lang': 'eng',
 'version': '20190520',
 'provider_name': "Who's On First",
 'id': 'carto-do-public-data.whos_on_first.geography_glo_geojson_20190520'}

Now we are going to perform a query to the table in order to retrieve the different geometries given a city name and a country ISO Alpha-2 code. As we are looking for city boundaries, we can also limit our search to the placetype = 'locality' if we find that this is the specific type of place for our needs.

In [10]:
city_name = "'Madrid'"
country_code = "'ES'"
placetype = "'locality'"

sql_query = f"SELECT * FROM $geography$ WHERE name = {city_name} AND country = {country_code}"

wof_geojson_filtered = wof_geojson.to_dataframe(sql_query=sql_query)

wof_geojson_filtered

,geoid,id,body,name,country,parent_id,is_current,placetype,geometry_type,bbox,geom,lastmodified,lastmodified_timestamp
0,404338863,404338863,"{""id"": 404338863, ""type"": ""Feature"", ""properti...",Madrid,ES,85682783,0,localadmin,Polygon,"POLYGON((-3.51823494 40.31206394, -3.51823494 ...","POLYGON ((-3.88558 40.57446, -3.88560 40.57442...",1513267506,2017-12-14 16:05:06+00:00
1,101748283,101748283,"{""id"": 101748283, ""type"": ""Feature"", ""properti...",Madrid,ES,85682783,1,locality,MultiPolygon,"POLYGON((-3.5180508952556 40.312064309035, -3....","POLYGON ((-3.77456 40.40031, -3.77456 40.40026...",1536881193,2018-09-13 23:26:33+00:00
2,85682783,85682783,"{""id"": 85682783, ""type"": ""Feature"", ""propertie...",Madrid,ES,404227387,1,region,MultiPolygon,"POLYGON((-3.05298331 39.88471951, -3.05298331 ...","MULTIPOLYGON (((-4.31951 40.64764, -4.31873 40...",1553814873,2019-03-28 23:14:33+00:00


It may happen that we find that there are more than one locality with the same name. In order to select the right polygon, we can build a map with a category widget that will allow us to decide for the specific geometry that we are looking for.

In [11]:
wof_geojson_filtered['geoid_str'] = wof_geojson_filtered['geoid'].astype(str) 
Map(
    Layer(
      wof_geojson_filtered, # where the data comes from
      color_category_style('geoid_str',palette='Vivid',opacity=0.6,stroke_width=0.2),
      widgets=[category_widget('geoid_str','Select geoid to visualize')],
      popup_hover=[popup_element('geoid','geoid'),
                        popup_element('name','name'),
                        popup_element('placetype','placetype')],
      legends=color_category_legend('Geoid'),
      geom_col='geom', #the name of the column on the query that has a GEOGRAPHY data
      encode_data = False  
  )
)

Once we know which geometry (i.e. polygon of city boundaries) is the right one for our tests, we should copy/note its associated geoid.

As in this example we want to filter the data for Madrid, we will use geoid = '	101748283'.

In [12]:
"""Helper function for downloading only the data within the area (geometry) of interest
    Args:
        do_dataset: DO Dataset you'd like to download for a specific area of interest
        do_geom_dataset: DO Dataset containing the geometry you'd like to use as filter (your area of interest)
        target_geoid: geoid of the geometry you'd like to use as filter (your area of interest)
"""
def filter_data(do_dataset, do_geom_dataset, target_geoid):
    do_geom_dataset_id=do_geom_dataset.id
    sql_query = f"""WITH do_geom AS (
      SELECT geom
      FROM `{do_geom_dataset_id}`
      WHERE geoid = '{target_geoid}')

      SELECT do_d.* FROM $dataset$ do_d, do_geom WHERE ST_Intersects(do_d.geom, do_geom.geom)"""
    filtered_data = do_dataset.to_dataframe(sql_query = sql_query)
    return filtered_data

In [13]:
SpatialFeatures_esp_qk15_madrid = filter_data(SpatialFeatures_esp_qk15,wof_geojson,'101748283')
SpatialFeatures_esp_qk15_madrid.head()

,geoid,do_date,country_iso,country_iso_a3,population,female,male,female_under_1,female_1_to_4,female_5_to_9,female_10_to_14,female_15_to_19,female_20_to_24,female_25_to_29,female_30_to_34,female_35_to_39,female_40_to_44,female_45_to_49,female_50_to_54,female_55_to_59,female_60_to_64,female_65_to_69,female_70_to_74,female_75_to_79,female_80_and_over,male_under_1,male_1_to_4,male_5_to_9,male_10_to_14,male_15_to_19,male_20_to_24,male_25_to_29,male_30_to_34,male_35_to_39,male_40_to_44,male_45_to_49,male_50_to_54,male_55_to_59,male_60_to_64,male_65_to_69,male_70_to_74,male_75_to_79,male_80_and_over,retail,education,financial,food_drink,healthcare,leisure,tourism,transportation,geom
0,33111012103312,2020-01-01,Spain,ESP,9337.627117,4881.428819,4456.198031,41.766095,163.672177,221.137283,245.566234,231.080698,227.383101,245.334472,263.994076,320.650538,390.419935,389.567041,377.063165,353.968339,314.816339,266.407605,247.312870,205.064303,376.224547,44.155446,173.006394,234.309353,258.144348,243.008669,228.511522,234.622352,253.541816,313.944129,384.364319,377.072623,351.233350,316.763072,269.588137,222.433800,200.236423,148.929341,202.332936,3,0,0,1,0,0,0,10,"POLYGON ((-3.70239 40.32980, -3.70239 40.33817..."
1,33111012112020,2020-01-01,Spain,ESP,12524.983924,6547.682718,5977.301260,56.022764,219.540946,296.621506,329.389162,309.959063,304.999293,329.078293,354.107280,430.103163,523.688038,522.544012,505.771966,474.793871,422.277495,357.344649,331.732017,275.062080,504.647118,59.227708,232.061352,314.289800,346.260753,325.958592,306.512897,314.709633,340.087184,421.107543,515.565376,505.784677,471.125289,424.888722,361.610845,298.360577,268.586231,199.765704,271.398376,4,6,3,10,6,18,0,73,"POLYGON ((-3.68042 40.35492, -3.68042 40.36329..."
2,33111010323312,2020-01-01,Spain,ESP,12421.076833,6493.363030,5927.713505,55.557999,217.719627,294.160728,326.656544,307.387631,302.469019,326.348264,351.169585,426.535027,519.343525,518.208965,501.576084,470.854971,418.774260,354.380114,328.979961,272.780164,500.460560,58.736354,230.136176,311.682449,343.388167,323.254432,303.970064,312.098805,337.265824,417.614041,511.288227,501.588670,467.216838,421.363837,358.610911,295.885376,266.358035,198.108446,269.146852,27,2,0,31,8,9,4,36,"POLYGON ((-3.70239 40.46367, -3.70239 40.47202..."
3,33111010330212,2020-01-01,Spain,ESP,2254.660193,1178.668140,1075.992060,10.084827,39.520229,53.395733,59.294337,55.796666,54.903846,59.238377,63.743920,77.424168,94.270665,94.064726,91.045543,85.469076,76.015445,64.326690,59.716083,49.514756,90.843051,10.661759,41.774066,56.576256,62.331445,58.676793,55.176313,56.651833,61.220123,75.804843,92.808480,91.047825,84.808686,76.485502,65.094659,53.708791,48.349018,35.960428,48.855241,0,0,0,0,0,0,0,0,"POLYGON ((-3.65845 40.53050, -3.65845 40.53885..."
4,33111012101110,2020-01-01,Spain,ESP,15010.814752,7847.199883,7163.615039,67.141591,263.113193,355.491909,394.762959,371.476565,365.532447,394.390390,424.386867,515.465653,627.624291,626.253203,606.152427,569.026092,506.086817,428.266773,397.570788,329.653602,604.804319,70.982620,278.118529,376.666836,414.983067,390.651518,367.346454,377.170004,407.584217,504.684685,617.889516,606.167654,564.629409,509.216307,433.379674,357.576148,321.892486,239.413157,325.262757,83,7,31,61,26,4,3,97,"POLYGON ((-3.70239 40.43859, -3.70239 40.44695..."


In [14]:
Map(
    Layer(
      SpatialFeatures_esp_qk15_madrid, # where the data comes from
      color_continuous_style('population', palette=palettes.magenta,opacity=0.75,stroke_width=0.5),
      widgets=[formula_widget('population','sum','Total Population'),histogram_widget('population','Population by cell')],
      popup_hover=[popup_element('population','Population')],
      geom_col='geom', #the name of the column on the query that has a GEOGRAPHY data
      encode_data = False  
  )
)